# 04.  TRANSFORMER

1. Intro
2. Decoder
3. Transformer
4. Learning
5. Tasks
6. LLMs
7. Exercise
8. References

# 1. Introduction

Seq2seq --- это семейство подходов для задач преобразования последовательностей.

In [1]:
from IPython.display import IFrame

IFrame(src='http://jalammar.github.io/images/seq2seq_3.mp4', width=800, height=None)

В 2017 в работе [Vaswani et al - Attention Is All You Need](https://arxiv.org/abs/1706.03762) была предложена архитектура Трансформер (Google Brain).

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/model-min.png)

#### More
- [Vaswani et al - Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- [Jay Allamar - The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [Huang et al - The Annotated Transformer](http://nlp.seas.harvard.edu/annotated-transformer/)
- [Lena Voita - Seq2seq and Attention](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html)
- [Brandon Rohrer - Transformers from Scratch](https://e2eml.school/transformers.html)
- [Peter Bloem - Transformers from Scratch](https://peterbloem.nl/blog/transformers)

# 2. Decoder (GPT)

Общая схема работы декодера:

![](http://jalammar.github.io/images/gpt2/gpt2-self-attention-example-2.png)

Подслой self-attention в декодере модифицирован:
- добавлены маски для предотвращения посещения последующих позиций.

![](res/05_decoder.png)

#### Masked Self-Attention

Сравнение self-attention и masked self-attention:

![](http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png)

Такое маскирование гарантирует, что прогнозы для позиции $i$ могут зависеть только от известных выходов в позициях, меньших $i$. Используется треугольная маска:

![](https://peterbloem.nl/files/transformers/masked-attention.svg)

#### Generation

Для преобразования выходов декодеров в предсказанные вероятности следующего токена используется обучаемое линейное преобразование и softmax.

![](https://jalammar.github.io/images/t/transformer_decoder_output_softmax.png)

На выходе декодера мы получаем вектор.
Этот вектор проходит через линейный слой (fully connected neural network), за которым следует softmax.

Линейный слой проецирует выход декодера в гораздо больший вектор (logits-вектор).
Размер этого вектор --- это размер словаря токенов.

Затем слой softmax превращает координаты в вероятности.
Выбирается наиболее вероятный токен.

#### Словарь и logits

![](http://jalammar.github.io/images/gpt2/gpt2-output-scores-2.png)

#### Autoregression

![](https://habrastorage.org/getpro/habr/upload_files/80e/243/698/80e24369887bf050a35ece72a3e161b5.gif)

[comment]: <> (http://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif)

#### Greedy search

![](https://huggingface.co/blog/assets/02_how-to-generate/greedy_search.png)

#### Beam search

![](https://huggingface.co/blog/assets/02_how-to-generate/beam_search.png)

#### More:

- [nn.labml.ai - GPT](https://nn.labml.ai/transformers/gpt/index.html)
- [Jay Alammar - The Illustrated GPT-2](http://jalammar.github.io/illustrated-gpt2/)

# 3. Transformer

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/model-min.png)

Декодер трансформера состоит из $N$ последовательных одинаковых слоёв (например, $N = 6$).
Кроме аналогичных двух подслоёв добавляется третий подслой,
- который реализует multi-head attention над выходами с энкодера.

#### Развёрнутая схема

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

#### Работа декодера

![](https://jalammar.github.io/images/t/transformer_decoding_1.gif)

#### Работа энкодера

![](https://jalammar.github.io/images/t/transformer_decoding_2.gif)

Примеры:
- GPT
- PaLM

#### More:
- [Transformer Encoder and Decoder Models](https://nn.labml.ai/transformers/models.html)
- [Patrick von Platen - How to generate text: using different decoding methods for language generation with Transformers](https://huggingface.co/blog/how-to-generate)

# 4. Learning

- обучать с нуля (*from scratch*)
- использовать модели, которые были обучены на других (но связанных с текущей) задачах (*transfer learning*)

Если обучать модель полностью с нуля, то потребуется много рамеченных данных. Где взять?

Идея:
- давайте найдём подходящую промежуточную задачу со свойствами:
    - связана с исходной задачей
    - легко найти для неё много размеченных данных
- обучимся на ней (*pretraining*)
- дообучим модель (*finetuning*) на нашей задачи (*downstream*-задача), используя меньший датасет

| обучение | тип задачи        | назначение          | таргеты        | разметка        |
|----------|-------------------|---------------------|----------------|-----------------|
| pretrain | pretrain-задача   | моделирование языка | синтетические  | self-supervised |
| finetune | downstream-задача | полезная задача     | реальные       | supervised      |

## Pretraining

Обучение модели с нуля промежуточной задаче. Особенности:
- требуется много данных
- дорого

Выбор промежуточной задачи:
- связь с конечной задачей, например, если NLP, то в качестве такой задачи подойдёт *задача моделирования языка*
- наличие большого количества размеченных данных.

Разметка с помощью экспертов:
- долго,
- дорого.

Поэтому на этом этапе используется автоматическая разметка (self-supervised), т.е. разметка, которую можно получить без участия человека:
- быстро,
- дёшево.

#### Causal language modeling (CLM)

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/causal_modeling.svg)

- слева направо
- авторегрессионно
- GPT (decoder)
- Prefix language modeling (PrefixLM)

#### Masked language modeling (MLM)

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/masked_modeling.svg)

- BERT (encoder)

#### Span corruption

![](https://user-images.githubusercontent.com/6536835/116129345-4b306700-a6ca-11eb-9acd-a14aa2b8d115.png)

- T5 (encoder-decoder)

#### UL2

- R-denoising (regular denoising): standard span corruption objective --- short spans & low corruption
- S-denoising (sequential denoising): strictly follows sequence order --- sequential denoising / prefix language modeling
- X-denoising (extreme denoising): extreme span lengths and corruption rates

![](res/05_denoising.png)

Подробнее:
- [Tay et al - UL2: Unifying Language Learning Paradigms](https://arxiv.org/abs/2205.05131)

## Finetuning (SFT)


### Датасеты / бенчмарки

- SQuAD ---  Stanford Question Answering Dataset
- CoQA --- Conversational Question Answering
- (Super) GLUE --- General Language Understanding Evaluation benchmark
    - CoLA --- Corpus of Linguistic Acceptability
    - SST --- Stanford Sentiment Treebank
    - MRPC --- Microsoft Research Paraphrase Corpus
    - QQP --- Quora Question Pairs
    - MultiNLI --- Multi-Genre Natural Language Inference Corpus
    - RTE --- Recognizing Textual Entailment
    - etc

### Метрики

- Exact match (EM)
- F1 score
- Perplexity
- BLEU

Подробнее:
- [Resources and Benchmarks for NLP](https://slds-lmu.github.io/seminar_nlp_ss20/resources-and-benchmarks-for-nlp.html)

# 5. Tasks

Natural language processing (обработка естественного языка) --- различные задачи, связанные с обработкой текста.

#### Sentiment analysis

Анализ тональности текста, выявление эмоциональной окраски текста.

![](res/05_sentiment_analysis.png)

Например:
- [Pascual - Getting Started with Sentiment Analysis using Python](https://huggingface.co/blog/sentiment-analysis-python)

In [ ]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

#### Text classification

Классификация текстов или документов.

![](https://developers.google.com/static/machine-learning/guides/text-classification/images/TextClassificationExample.png)

Например:
- [developers.google.com - Text Classification](https://developers.google.com/machine-learning/guides/text-classification)

#### Named entity recognition (NER)

Идентификация и классификации сущностей (таких как люди, места и организации...) в тексте.

![](https://www.shaip.com/wp-content/uploads/2022/02/Blog_Named-Entity-Recognition-%E2%80%93-The-Concept-Types-Applications.jpg.webp)

Например:
- [bert-base-NER ](https://huggingface.co/dslim/bert-base-NER)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

#### Part-of-speech tagging (POS tagging, POST)

Определение, какой частью речи является каждое слово в предложении.

![](https://slideplayer.com/slide/6426089/22/images/2/Part-of-Speech+Tagging.jpg)

Например:
- [English Part-of-Speech Tagging in Flair](https://huggingface.co/flair/pos-english)

#### Question answering (QA)

Генерация ответов на вопросы. Вариации:
- запрос содержит ответ на вопрос (например, документ + вопрос к документу)
- запрос не содержит ответа на вопрос (необходимо использовать информацию, извлечённую на этапе обучения)

![](https://www.nlplanet.org/course-practical-nlp/_images/question_answering_11.png)
![](https://www.nlplanet.org/course-practical-nlp/_images/question_answering_21.png)

#### Text summarization

Генерация резюме текста (абстрактивная суммаризация и экстрактивная суммаризация).

![](https://assets-global.website-files.com/62ab5c229babcf02f79fbd7d/63bd8a1f20da862484184fdb_blog%20extractive%20-p-1600.png)

Подробнее:
- [Summarization](https://huggingface.co/docs/transformers/tasks/summarization)

#### (Mathematical) reasoning

Построение рассуждений, решение математических задач

![](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEgpHov9BD2yiBEDrAWZUQxDYWRIuofmpbWVJaJPDPrE-2BbT3B_15-R4n22yNnDVs_8Vkea-Y-ykOHaB6mCKwkLYkBDBoS1r8NX2u4KsCpNC53GAM_8seK6L_90CJCmhC4ML9SSVY03lErXDQd6Pp-ysGsANdvNcqur7lMARO7h4RtDtf6Y7UlNYuEjjQ/s1999/image5.png)

#### NLU vs NLG

Обычно выделяют:
- задачи генерации текста (NLG) --- происходит генерация ответа на естественном языке
- задачи понимания текста (NLU) --- происходит генерация (или дополнение) ответа в рамках некоторого формата

<img src="https://datasolut.com/wp-content/uploads/2021/05/NLP-100.jpg.webp" alt="drawing" width="800"/>

#### Модели энкодеры

![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/encoder_decoder/Encoder_block.png)

Примеры:
- BERT
- RoBERTa

#### Модели декодеры

![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/encoder_decoder/encoder_decoder_detail.png)

#### Модели энкодер-декодеры

![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/encoder_decoder/EncoderDecoder.png)

[comment]: <> (https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/encoder_decoder/EncoderDecoder_step_by_step.png)


Примеры:
- BART
- T5

#### Сравнение

|                |   NLU      | NLG        |
|----------------|------------|------------|
| encoder-only   | optimal    | poor       |
| decoder-only   | poor       | optimal    |
| encoder-decoder| suboptimal | suboptimal |

Например, в ML4SE:

![](https://github.com/microsoft/CodeXGLUE/raw/main/baselines.jpg)

[comment]: <> (https://github.com/microsoft/CodeXGLUE/raw/main/tasks.jpg)

А если необходимо отличное качество одновременно в NLU и NLG?
![CodeT5+](res/05_codet5plus.png)

Подробнее:
- [Wang et al - CodeT5+: Open Code Large Language Models for Code Understanding and Generation](https://arxiv.org/abs/2305.07922)

# 6. LLMs

### Scaling laws

![](res/05_scaling_laws.png)

### Large models

![](https://cdn.builder.io/api/v1/image/assets%2Fe0438815ba51486bbb6a202747122d4b%2F894dade70d724952bfad3956c599865a)

#### In-context learning

![](http://ai.stanford.edu/blog/assets/img/posts/2022-08-01-understanding-incontext/images/image13.gif)

Подробнее:

- [Kaplan et al - Scaling Laws for Neural Language Models](https://arxiv.org/abs/2001.08361v1)
- [Wei et al - Emergent Abilities of Large Language Models](https://arxiv.org/abs/2206.07682)
- [Wei et al - Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/abs/2201.11903)
- [Why Can GPT Learn In-Context? Language Models Implicitly Perform Gradient Descent as Meta-Optimizers](https://arxiv.org/abs/2212.10559)

# 7. Exercise

Реализовать Decoder с нуля на PyTorch. Быть готовыми ответить на вопросы по коду.

# 8. References

- [Lilian Weng - The Transformer Family](https://lilianweng.github.io/posts/2023-01-27-the-transformer-family-v2/)
- [nn.labml.ai - Paper Implementations](https://nn.labml.ai/)
- [Andrej Karpathy - nanoGPT](https://github.com/karpathy/nanoGPT)
- [Andrej Karpathy - minGPT](https://github.com/karpathy/minGPT)
- [pytorch - NLP from Scratch](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)
- [nn.labml.ai - GPT](https://nn.labml.ai/transformers/gpt/index.html)